In [1]:
import os
import sys
import pickle
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
sys.path.append(os.path.dirname(os.path.abspath('.')))
import configs
from datasets import TruthfulQADataset, DishonestQADataset, AmongUsDataset, RolePlayingDataset, RepEngDataset, ApolloProbeDataset
from probes import LinearProbe

from configs import config_phi4

In [2]:
dataset_name = 'RolePlayingDataset'
config = config_phi4
model, tokenizer, device = None, None, 'cpu'
EXPT_NAME: str = "2025-02-01_phi_phi_100_games_v3"
dataset = eval(f"{dataset_name}")(config, model=model, tokenizer=tokenizer, device=device, expt_name=EXPT_NAME, test_split=0.2)

In [3]:
train_loader = dataset.get_train(batch_size=32, num_tokens=10, chunk_idx=0)

In [4]:
probe = LinearProbe(input_dim=dataset.activation_size, device=device)

print(f'Training probe on {len(train_loader)} batches and {len(train_loader.dataset)} samples.')

Training probe on 93 batches and 2960 samples.


In [5]:
probe.fit(train_loader, epochs=10)

Epoch 1: Train Loss = 0.6060, Train Acc = 0.6797
Epoch 2: Train Loss = 0.3929, Train Acc = 0.8382
Epoch 3: Train Loss = 0.3185, Train Acc = 0.8733
Epoch 4: Train Loss = 0.2633, Train Acc = 0.9159
Epoch 5: Train Loss = 0.2248, Train Acc = 0.9389
Epoch 6: Train Loss = 0.1910, Train Acc = 0.9524
Epoch 7: Train Loss = 0.1665, Train Acc = 0.9669
Epoch 8: Train Loss = 0.1454, Train Acc = 0.9801
Epoch 9: Train Loss = 0.1300, Train Acc = 0.9828
Epoch 10: Train Loss = 0.1131, Train Acc = 0.9902
Final Train Acc: 0.9902


In [6]:
checkpoint_path = f'checkpoints/{dataset_name}_probe_{config["short_name"]}.pkl'
with open(checkpoint_path, 'wb') as f:
    pickle.dump(probe, f)
    print(f"Probe saved to {checkpoint_path}")

Probe saved to checkpoints/RolePlayingDataset_probe_phi4.pkl
